In [10]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 6.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=1c61908da8eb7da6e1ac9aadb3963c0b1752d1049dee88fd598043d2589f1910
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=91231211322e7cf3d3db0e0be6399fb33519afa11932c489c568b74c6d9d328e
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
!pip install tensorflow
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [11]:
def build_model(hp):
  model = keras.Sequential([
     keras.layers.Conv2D(
         filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
         kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
         activation='relu',
         input_shape=(28,28,1)
     ),
     keras.layers.Conv2D(
         filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
         kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
         activation='relu'
     ),
     keras.layers.Flatten(),
     keras.layers.Dense(
         units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
         activation='relu'
     ),
     keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model
         


In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [17]:
tuner_search=RandomSearch(build_model,
                         objective='val_accuracy',
                         max_trials=3,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 2 Complete [00h 17m 22s]
val_accuracy: 0.8758333325386047

Best val_accuracy So Far: 0.9125000238418579
Total elapsed time: 00h 32m 55s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |64                |112               
conv_1_kernel     |3                 |3                 
conv_2_filter     |48                |48                
conv_2_kernel     |5                 |3                 
dense_1_units     |64                |128               
learning_rate     |0.01              |0.001             

Epoch 1/3
1688/1688 [==============================] - 345s 204ms/step - loss: 0.5050 - accuracy: 0.8264 - val_loss: 0.4060 - val_accuracy: 0.8453
Epoch 2/3
 749/1688 [============>.................] - ETA: 3:08 - loss: 0.3686 - accuracy: 0.8634

KeyboardInterrupt: ignored

In [15]:
model=tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        48432     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3539072   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 3,589,914
Trainable params: 3,589,914
Non-trainable params: 0
_________________________________________________________________
